In [39]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

In [3]:
df = pd.read_csv("https://jedha-final-project-jrat.s3.amazonaws.com/datameteo_france_1950-2022_clean_02.csv")

In [4]:
df_sample = df[df["NUM_POSTE"]==28070001]

In [23]:
df_sample

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMM,Year,Month,precip_cumul_mensu,temp_mean_mensu,...,vent_speed_inst_max_mensu,vent_dir_inst,vent_nbjour_inst_speedsup10ms,vent_speed_10mn_max_mensu,vent_dir_10mn,vent_nbjour_10mn_speedsup10ms,departement_num,departement_name,region,date
113156,28070001,CHARTRES,48.4605,1.501167,155,1950-01-01,1950,1,20.6,4.7,...,54.0,NaN,10.0,NaN,NaN,NaN,28,Eure-et-Loir,CENTRE VAL DE LOIRE,1950-01-31
113157,28070001,CHARTRES,48.4605,1.501167,155,1950-02-01,1950,2,56.8,10.1,...,72.0,NaN,20.0,NaN,NaN,NaN,28,Eure-et-Loir,CENTRE VAL DE LOIRE,1950-02-28
113158,28070001,CHARTRES,48.4605,1.501167,155,1950-03-01,1950,3,24.4,12.1,...,64.8,NaN,11.0,NaN,NaN,NaN,28,Eure-et-Loir,CENTRE VAL DE LOIRE,1950-03-31
113159,28070001,CHARTRES,48.4605,1.501167,155,1950-04-01,1950,4,52.8,13.1,...,68.4,NaN,20.0,NaN,NaN,NaN,28,Eure-et-Loir,CENTRE VAL DE LOIRE,1950-04-30
113160,28070001,CHARTRES,48.4605,1.501167,155,1950-05-01,1950,5,65.6,19.2,...,72.0,NaN,12.0,NaN,NaN,NaN,28,Eure-et-Loir,CENTRE VAL DE LOIRE,1950-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114027,28070001,CHARTRES,48.4605,1.501167,155,2022-08-01,2022,8,41.4,28.4,...,51.5,230.0,12.0,23.8,80.0,0.0,28,Eure-et-Loir,CENTRE VAL DE LOIRE,2022-08-31
114028,28070001,CHARTRES,48.4605,1.501167,155,2022-09-01,2022,9,77.1,21.3,...,60.5,130.0,9.0,33.5,130.0,0.0,28,Eure-et-Loir,CENTRE VAL DE LOIRE,2022-09-30
114029,28070001,CHARTRES,48.4605,1.501167,155,2022-10-01,2022,10,51.7,19.8,...,73.4,230.0,11.0,36.4,210.0,1.0,28,Eure-et-Loir,CENTRE VAL DE LOIRE,2022-10-31
114030,28070001,CHARTRES,48.4605,1.501167,155,2022-11-01,2022,11,54.9,12.4,...,66.6,240.0,13.0,32.4,210.0,0.0,28,Eure-et-Loir,CENTRE VAL DE LOIRE,2022-11-30


In [20]:
df_sample["AAAAMM"] = pd.to_datetime(df_sample["AAAAMM"])


C:\Users\alexm\AppData\Local\Temp\ipykernel_14580\1327262823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["AAAAMM"] = pd.to_datetime(df_sample["AAAAMM"])


Timestamp('1950-02-01 00:00:00')

In [22]:
# obtenir le dernier jour du mois pour l'ajouter à la date (important pour la saisonnalité)

def last_day_of_month(aaaamm):
    year = aaaamm.year
    month = aaaamm.month
    last_day = monthrange(year, month)[1]
    return f"{year}-{month:02d}-{last_day:02d}"

df_sample["date"] = df_sample["AAAAMM"].apply(last_day_of_month) 





C:\Users\alexm\AppData\Local\Temp\ipykernel_14580\2525275840.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["date"] = df_sample["AAAAMM"].apply(last_day_of_month)


In [24]:
df_sample["date"] = pd.to_datetime(df_sample["date"])
df_sample["year"] = df_sample["date"].dt.year
df_sample["month"] = df_sample["date"].dt.month


C:\Users\alexm\AppData\Local\Temp\ipykernel_14580\3732484278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["date"] = pd.to_datetime(df_sample["date"])
C:\Users\alexm\AppData\Local\Temp\ipykernel_14580\3732484278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["year"] = df_sample["date"].dt.year
C:\Users\alexm\AppData\Local\Temp\ipykernel_14580\3732484278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [25]:
# Pour modèle sarimax, il faut définir l'index en date et indiquer la fréquence de saisonnalité
df_sample.set_index("date", inplace=True)


In [26]:
df_sample = df_sample.asfreq('ME')  # 'M' pour mensuel

In [38]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876 entries, 1950-01-31 to 2022-12-31
Freq: ME
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   NUM_POSTE                      876 non-null    int64         
 1   NOM_USUEL                      876 non-null    object        
 2   LAT                            876 non-null    float64       
 3   LON                            876 non-null    float64       
 4   ALTI                           876 non-null    int64         
 5   AAAAMM                         876 non-null    datetime64[ns]
 6   Year                           876 non-null    int64         
 7   Month                          876 non-null    int64         
 8   precip_cumul_mensu             876 non-null    float64       
 9   temp_mean_mensu                876 non-null    float64       
 10  temp_max_mensu                 876 non-null    float64    

In [27]:
to_drop = [col for col in df_sample.columns if df_sample[col].isna().any() == True]

In [28]:
df_sample = df_sample.drop(to_drop,axis=1)

In [29]:
df_sample["year"].describe()

count     876.000000
mean     1986.000000
std        21.083345
min      1950.000000
25%      1968.000000
50%      1986.000000
75%      2004.000000
max      2022.000000
Name: year, dtype: float64

In [30]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876 entries, 1950-01-31 to 2022-12-31
Freq: ME
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   NUM_POSTE                      876 non-null    int64         
 1   NOM_USUEL                      876 non-null    object        
 2   LAT                            876 non-null    float64       
 3   LON                            876 non-null    float64       
 4   ALTI                           876 non-null    int64         
 5   AAAAMM                         876 non-null    datetime64[ns]
 6   Year                           876 non-null    int64         
 7   Month                          876 non-null    int64         
 8   precip_cumul_mensu             876 non-null    float64       
 9   temp_mean_mensu                876 non-null    float64       
 10  temp_max_mensu                 876 non-null    float64    

In [31]:
train = df_sample[df_sample["year"]<=2022-7]
test = df_sample[df_sample["year"]>2022-7]

In [32]:
to_keep = ["LON","LAT","ALTI"]

var_exog_train = train[to_keep]
y_train = train["vent_speed_inst_moy_mensu"]

var_exog_test = test[to_keep]
y_test = test["vent_speed_inst_moy_mensu"]

In [33]:
model = SARIMAX(y_train, exog=var_exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_model = model.fit(maxiter=1000)

In [34]:
# Prédiction sur 7 ans => 84 mois
forecast = sarima_model.get_forecast(steps=84,exog=var_exog_test)
predicted_values = forecast.predicted_mean

In [35]:
print(predicted_values)

2016-01-31    11.966387
2016-02-29    11.854666
2016-03-31    11.472959
2016-04-30    11.127650
2016-05-31    10.304425
                ...    
2022-08-31     8.414404
2022-09-30     8.639961
2022-10-31     9.184516
2022-11-30    10.004519
2022-12-31    11.010836
Freq: ME, Name: predicted_mean, Length: 84, dtype: float64


In [40]:
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
mae = mean_absolute_error(y_test, predicted_values)
mape = mean_absolute_percentage_error(y_test, predicted_values)


print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")

RMSE: 1.4408568612091903
MAE: 1.0837268631581611
MAPE: 0.11919462358392865
